# CUDA编程模型---矩阵乘法

#### 通过向量加法，我们已经学会了如何调用线程。接下来，我们来实践一下，如何利用Cuda处理矩阵。今天的课程将会介绍：
1. 二维矩阵的乘法
2. 如何分配线程和GPU存储单元

#### 1.矩阵乘法是科学计算和深度学习领域常见的操作，我们先来看一看CPU代码如何处理矩阵乘法

    void cpu_matrix_mult(int *h_a, int *h_b, int *h_result, int m, int n, int k) 
    {
        for (int i = 0; i < m; ++i) 
        {
            for (int j = 0; j < k; ++j) 
            {
                int tmp = 0.0;
                for (int h = 0; h < n; ++h) 
                {
                    tmp += h_a[i * n + h] * h_b[h * k + j];
                }
                h_result[i * k + j] = tmp;
            }
        }
    }

#### 2.这时，我们看到在CPU代码中，需要嵌套三个for循环，也就是说CPU的线程会一个接一个的求结果矩阵中的每一个数值，直到处理完所有数值。那么，我们在GPU中就可以申请很多个线程，每个线程来求结果矩阵中的一个数值，并同时完成。![matrix_mul](matrix_mul.png)

那么，首先我们要得到每一个执行线程，在Grid所有线程中的(x,y)坐标，如下图所示，即(Thread_x, Thread_y) 
![matrix_mul2](matrix_mul2.png)

![array_2to1.png](array_2to1.png)

也就是说，以上面的CPU代码为例，我们要让编号为(Thread_x, Thread_y)的线程读取a矩阵中的一行和b矩阵中的一列，然后把对应元素乘积并累加。

接下来，我们在[matrix_mul.cu](matrix_mul.cu)中完成这一过程，如果遇到麻烦，请参考[result1.cu](result1.cu)

修改Makefile文件，并编译

In [14]:
!make

/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80  matrix_mul.cu -o ./matrix_mul


执行，并查看结果是否正确

In [15]:
!./matrix_mul

计算和 CPU 基本一样 ~
23154685 


利用nvprof来查看程序性能

In [5]:
# !sudo /usr/local/cuda/bin/nvprof ./matrix_mul
!/usr/local/cuda/bin/nsys nvprof --print-api-trace ./matrix_mul

The --print-api-trace  switch is ignored by nsys.

计算和 CPU 基本一样 ~
Generating '/tmp/nsys-report-150b.qdstrm'
[1/7] [========================100%] report1.nsys-rep
[2/7] [========================100%] report1.sqlite
[3/7] Executing 'nvtxsum' stats report
SKIPPED: /mnt/CUDA_on_ARM/02_2.4/report1.sqlite does not contain NV Tools Extension (NVTX) data.
[4/7] Executing 'cudaapisum' stats report

CUDA API Statistics:

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)   Med (ns)  Min (ns)  Max (ns)   StdDev (ns)        Name      
 --------  ---------------  ---------  ----------  --------  --------  ---------  -----------  ----------------
     99.8        203584295          4  50896073.8    5950.5      4125  203568269  101781463.5  cudaMallocHost  
      0.1           181445          3     60481.7    4107.0      3025     174313      98582.3  cudaMalloc      
      0.1           180379          3     60126.3    7573.0      3709     169097      94391.1  cudaFree        
      0.0            6898

修改矩阵大小为1000*1000，并查看效果

In [6]:
!make

/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80  matrix_mul.cu -o ./matrix_mul


In [7]:
!/usr/local/cuda/bin/nsys nvprof --print-api-trace ./matrix_mul

The --print-api-trace  switch is ignored by nsys.

计算和 CPU 基本一样 ~
Generating '/tmp/nsys-report-bf85.qdstrm'
[1/7] [========================100%] report2.nsys-rep
[2/7] [========================100%] report2.sqlite
[3/7] Executing 'nvtxsum' stats report
SKIPPED: /mnt/CUDA_on_ARM/02_2.4/report2.sqlite does not contain NV Tools Extension (NVTX) data.
[4/7] Executing 'cudaapisum' stats report

CUDA API Statistics:

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)   Med (ns)   Min (ns)  Max (ns)   StdDev (ns)        Name      
 --------  ---------------  ---------  ----------  ---------  --------  ---------  -----------  ----------------
     54.3        205614597          4  51403649.3  1590566.0   1571659  200861806   99638771.6  cudaMallocHost  
     44.5        168206193          3  56068731.0   321519.0    318253  167566421   96559832.0  cudaFree        
      0.6          2083558          3    694519.3   364403.0    337782    1381373     594981.6  cudaMemcpy      
      0.5          1

课后作业：
1. 当我们能申请的线程数很少，远远不够的时候怎么办？
2. 修改[im2gray.cu](im2gray.cu), 完成将RGB图像转化为灰度图的程序. 如果遇到麻烦, 请参考[im2gray_result.cu](im2gray_result.cu)

In [11]:
# 这里根据自己的环境进行了修改。
!/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80 im2gray.cu -L /usr/lib/x86_64-linux-gnu/libopencv*.so -I /usr/include/opencv2 -o im2gray

In [12]:
!./im2gray